In [3]:
##Import required Google Earth Engine python packages and check if they work in python environment
import ee
ee.Initialize()
import geetools
import geemap
import os
from geemap import cartoee
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd 
from geetools import batch

In [4]:
#import the map module that allows for attaching images to an interactive map
Map = geemap.Map()

In [5]:
#Import the river boundary from the Google Earth Engine Server
#Call in river in from a vector file saved into Google Earth Engine
TN_River = ee.FeatureCollection("users/pjf927/TN_River_GERS_StudySite")
#Some function require geometry values to clip features
TN_RiverGeom = TN_River.geometry() 
#Generate a square boundary around the river study area
RiverBounds = TN_RiverGeom.bounds()

In [6]:
#Call in Landsat 5 Level 2, Collection 2, Tier 1 dataset 
LS8_SR = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(TN_River) #Filter only swath grids that cover the TN River Boundary
    #.select(['QA_PIXEL']) #Assign imagery bands new names
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'QA_PIXEL'],['Blue', 'Green', 'Red', 'QA_PIXEL']) #Assign imagery bands new names
    .filterDate('2013-03-18', '2023-01-21')
    #.filterDate('2009-05-28', '2014-05-27')
    #.select(['QA_PIXEL'])
    .sort('system:time_start') #Sort collection by acquisition time
)

#Get a count of all images filtered in the Landsat Surface Relectance Collection
LS8_count_raw = LS8_SR.size().getInfo()
print("Landsat 8 Images: ", LS8_count_raw)

Landsat 8 Images:  727


In [ ]:
def mergeByDate(imgCol):
    #Convert the image collection to a list.
    imgList = imgCol.toList(imgCol.size())
    
    # Driver function for mapping the unique dates
    def uniqueDriver(image):
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    uniqueDates = imgList.map(uniqueDriver).distinct()

    # Driver function for mapping the moasiacs
    def mosaicDriver(date):
        date = ee.Date(date)
        
        image = (imgCol
               .filterDate(date, date.advance(1, "day"))
               .mosaic())
        
        return image.set(
                        "system:time_start", date.millis(), 
                        "system:id", date.format("YYYY-MM-dd"),
                        "Date", date.format("YYYY-MM-dd"))
    
    mosaicImgList = uniqueDates.map(mosaicDriver)
    
    return ee.ImageCollection(mosaicImgList)

In [ ]:
LS8_SR_Mosaic = mergeByDate(LS8_SR)

#LS9_SR_Mosaic = mergeByDate(LS9_SR.map(lambda image: image.set('SPACECRAFT_ID', 'LANDSAT_9')))

In [ ]:
# Function declaration from Prof. Gunn
def getQABits(image, start, end, newName):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start, end+1):
       pattern += pow(2, i)
    # Return a single band image of the extracted QA bits, giving the band a new name.
    return image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)

In [ ]:
# A function to mask out cloudy pixels.
def cloud_mask(image):
    # Select the QA band.
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    cloud_only = getQABits(QA, 3,3, 'cloud_mask').eq(1)
    cloudiness = cloud_only.reduceRegion(
             reducer = ee.Reducer.mean(), 
             geometry = TN_RiverGeom, 
             scale = 30,
             maxPixels = 1e13).get("cloud_mask")
    percent = ee.Number(cloudiness).multiply(100)
    return image.set("CLOUD_COVER", percent)

In [ ]:
LS8_SR_Mask = LS8_SR_Mosaic.map(cloud_mask)

LS8_SR_Sensor = LS8_SR_Mask.map(lambda image: image.set('SPACECRAFT_ID', 'LANDSAT_8'))

In [ ]:
LS8_SR_Sensor = LS8_SR_Sensor.map(
    lambda img: img.set({"Date": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")})
)

In [ ]:
LS8_SR_Filtered = LS8_SR_Sensor.filterMetadata("CLOUD_COVER","less_than", 30)

In [ ]:
All_count_preprocess = LS8_SR_Filtered.size().getInfo()
print("Landsat Images: ", All_count_preprocess)

In [ ]:
dates = LS8_SR_Filtered.aggregate_array("Date").getInfo()
print("Dates in Imagecollection: ", dates)

In [ ]:
sensor = LS8_SR_Filtered.aggregate_array("SPACECRAFT_ID").getInfo()
print("sensor: ", sensor)

In [ ]:
# A function to mask out cloud shadow pixels.
def cloud_shadows(image):
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    # Get the internal_cloud_algorithm_flag bit.
    return getQABits(QA, 4,4, 'Cloud_shadows').eq(0)

# A function to mask out cloudy pixels.
def clouds(image):
    # Select the QA band.
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    return getQABits(QA, 3,3, 'Cloud').eq(0)
       
# A function to mask out cloud shadow pixels.
def water(image):
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    return getQABits(QA, 7,7, 'Water').neq(0)

In [ ]:
def maskClouds(image):
    cs = cloud_shadows(image)
    c = clouds(image)
    w = water(image)
    image = image.updateMask(cs).updateMask(c)
    return image.updateMask(w)

In [ ]:
#Clip out all pixels that are IN the TN River boundary and save as a new Image Collection using the lambda function
LS8_SR_Clip = LS8_SR_Filtered.map(lambda image: image.clip(TN_River))

In [ ]:
#Apply each equation to the clipped pixels that are in the TN River boundary
LS8_SR_Clip_Water = LS8_SR_Clip.map(maskClouds)

In [ ]:
#Create fucntion that calcualtes the Quantitative index 
def QUANT(image):
    red = image.select('Red') #Create a variable that selects the red band
    #Run the '2677.2 * (pow(Red, 1.856))' equation
    scale = red.multiply(0.0000275).add(-0.2)
    quant = (((scale.pow(1.856)).multiply(2677.2)).toFloat()).rename('QUANT')
    #return quant
    return quant

In [ ]:
LS8_SR_Clip_Water_Quant = LS8_SR_Clip_Water.map(QUANT)

In [ ]:
#Create a list that collects that orders the images so they can be mapped to the dates
LS8_SR_Clip_Water_Quant_List = LS8_SR_Clip_Water_Quant.toList(LS8_SR_Clip_Water_Quant.size())

In [ ]:
#QUANT iterate images to add map to layer, export to local directory, and export to Google Drive
for index in range(0, All_count_preprocess):
    image = ee.Image(LS8_SR_Clip_Water_Quant_List.get(index))
    layer_name = "QUANT_" + str(dates[index]) + "_" + str(sensor[index])
    #Map.addLayer(image, ndssiVis, layer_name, False)
    #QUANT_file = os.path.join(r'D:\Thesis\ASPRS\GEE_Assessment\Data\Rasters\Quant_Test', layer_name + ".tif")
    #geemap.ee_export_image(image, filename = QUANT_file, scale = 30, region = TN_RiverGeom, file_per_band = True)
    geemap.ee_export_image_to_drive(image, description = layer_name, folder = 'QUANT/2013-03-18_2023-01-21', region = TN_RiverGeom, scale = 30)

In [ ]:
#Clip out all pixels that are IN the TN River boundary and save as a new Image Collection using the lambda function
LS8_SR_True = LS8_SR_Filtered.map(lambda image: image.clip(RiverBounds))

In [ ]:
#Applies scaling factors so images are better looking for displaying it graphs
def applyScaleFactors(image):
  opticalBands = image.select(['Blue', 'Green', 'Red']).multiply(0.0000275).add(-0.2)
  return image.addBands(opticalBands, None, True) 

In [ ]:
#Applies the scaling factor to the image collection that has pixels AROUND the TN River boundary
LS8_SR_True_Bounds_Scale = LS8_SR_True.map(applyScaleFactors)

In [ ]:
LS8_SR_True_List = LS8_SR_True.toList(LS8_SR_True.size())

In [ ]:
#True color iterate images to add map to layer, export to local directory, and export to Google Drive
for index in range(0, All_count_preprocess):
    image = ee.Image(LS8_SR_True_List.get(index))
    layer_name = "True_" + str(dates[index]) + "_" + str(sensor[index])
    #Map.addLayer(image, trueVis, layer_name, False)
    #filename = os.path.join(r'D:\Thesis\ASPRS\GEE_Assessment\Data\Rasters\True_Test', layer_name + ".tif")
    #geemap.ee_export_image(image, filename = filename, scale = 30, region = TN_RiverGeom, file_per_band = False)
    geemap.ee_export_image_to_drive(image, description = layer_name, folder = 'True/2013-03-18_2023-01-21', region = TN_RiverGeom, scale = 30)